In [1]:
import numpy as np
import time
import os
import csv
import copy
import wave
import cv2
import matplotlib.pyplot as plt

from PIL import Image
from IPython.core.display import HTML

In [2]:
def relu(x):
    return np.maximum(x, 0)

def relu_derv(y):
    return np.sign(y)